# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 13 2023 3:49PM,264703,19,ADV80021057,"AdvaGen Pharma, Ltd",Released
1,Jun 13 2023 3:48PM,264702,10,Eme-113183,Emerginnova,Released
2,Jun 13 2023 3:47PM,264701,10,SONSB0002228,"Nextsource Biotechnology, LLC",Released
3,Jun 13 2023 3:39PM,264700,10,MSP222675,"Methapharm, Inc.",Released
4,Jun 13 2023 3:39PM,264699,10,MSP222679,Methapharm-G,Released
5,Jun 13 2023 3:38PM,264698,10,MSP222676-Hold_Rx(AL),Methapharm-G,Released
6,Jun 13 2023 3:35PM,264697,10,Med-9569770,MedStone Pharma LLC,Released
7,Jun 13 2023 3:35PM,264697,10,Next-9569772,MedStone Pharma LLC,Released
8,Jun 13 2023 3:04PM,264695,10,MSP222666,"Methapharm, Inc.",Released
9,Jun 13 2023 3:04PM,264695,10,MSP222667,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,264699,Released,1
19,264700,Released,1
20,264701,Released,1
21,264702,Released,1
22,264703,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
264699,NaN,NaN,1.0
264700,NaN,NaN,1.0
264701,NaN,NaN,1.0
264702,NaN,NaN,1.0
264703,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264654,14.0,7.0,5.0
264664,0.0,0.0,1.0
264674,0.0,0.0,6.0
264675,0.0,23.0,6.0
264676,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264654,14,7,5
264664,0,0,1
264674,0,0,6
264675,0,23,6
264676,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264654,14,7,5
1,264664,0,0,1
2,264674,0,0,6
3,264675,0,23,6
4,264676,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264654,14,7,5
1,264664,,,1
2,264674,,,6
3,264675,,23,6
4,264676,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 13 2023 3:49PM,264703,19,"AdvaGen Pharma, Ltd"
1,Jun 13 2023 3:48PM,264702,10,Emerginnova
2,Jun 13 2023 3:47PM,264701,10,"Nextsource Biotechnology, LLC"
3,Jun 13 2023 3:39PM,264700,10,"Methapharm, Inc."
4,Jun 13 2023 3:39PM,264699,10,Methapharm-G
5,Jun 13 2023 3:38PM,264698,10,Methapharm-G
6,Jun 13 2023 3:35PM,264697,10,MedStone Pharma LLC
8,Jun 13 2023 3:04PM,264695,10,"Methapharm, Inc."
12,Jun 13 2023 3:03PM,264694,10,Methapharm-G
13,Jun 13 2023 2:28PM,264687,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 13 2023 3:49PM,264703,19,"AdvaGen Pharma, Ltd",,,1
1,Jun 13 2023 3:48PM,264702,10,Emerginnova,,,1
2,Jun 13 2023 3:47PM,264701,10,"Nextsource Biotechnology, LLC",,,1
3,Jun 13 2023 3:39PM,264700,10,"Methapharm, Inc.",,,1
4,Jun 13 2023 3:39PM,264699,10,Methapharm-G,,,1
5,Jun 13 2023 3:38PM,264698,10,Methapharm-G,,,1
6,Jun 13 2023 3:35PM,264697,10,MedStone Pharma LLC,,,2
7,Jun 13 2023 3:04PM,264695,10,"Methapharm, Inc.",,,4
8,Jun 13 2023 3:03PM,264694,10,Methapharm-G,,,1
9,Jun 13 2023 2:28PM,264687,10,Emerginnova,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 3:49PM,264703,19,"AdvaGen Pharma, Ltd",1,,
1,Jun 13 2023 3:48PM,264702,10,Emerginnova,1,,
2,Jun 13 2023 3:47PM,264701,10,"Nextsource Biotechnology, LLC",1,,
3,Jun 13 2023 3:39PM,264700,10,"Methapharm, Inc.",1,,
4,Jun 13 2023 3:39PM,264699,10,Methapharm-G,1,,
5,Jun 13 2023 3:38PM,264698,10,Methapharm-G,1,,
6,Jun 13 2023 3:35PM,264697,10,MedStone Pharma LLC,2,,
7,Jun 13 2023 3:04PM,264695,10,"Methapharm, Inc.",4,,
8,Jun 13 2023 3:03PM,264694,10,Methapharm-G,1,,
9,Jun 13 2023 2:28PM,264687,10,Emerginnova,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 3:49PM,264703,19,"AdvaGen Pharma, Ltd",1,,
1,Jun 13 2023 3:48PM,264702,10,Emerginnova,1,,
2,Jun 13 2023 3:47PM,264701,10,"Nextsource Biotechnology, LLC",1,,
3,Jun 13 2023 3:39PM,264700,10,"Methapharm, Inc.",1,,
4,Jun 13 2023 3:39PM,264699,10,Methapharm-G,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 3:49PM,264703,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
1,Jun 13 2023 3:48PM,264702,10,Emerginnova,1.0,NaN,NaN
2,Jun 13 2023 3:47PM,264701,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
3,Jun 13 2023 3:39PM,264700,10,"Methapharm, Inc.",1.0,NaN,NaN
4,Jun 13 2023 3:39PM,264699,10,Methapharm-G,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 3:49PM,264703,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
1,Jun 13 2023 3:48PM,264702,10,Emerginnova,1.0,0.0,0.0
2,Jun 13 2023 3:47PM,264701,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
3,Jun 13 2023 3:39PM,264700,10,"Methapharm, Inc.",1.0,0.0,0.0
4,Jun 13 2023 3:39PM,264699,10,Methapharm-G,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3440990,31.0,1.0,0.0
15,264675,6.0,23.0,0.0
19,1058725,8.0,7.0,14.0
20,529349,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3440990,31.0,1.0,0.0
1,15,264675,6.0,23.0,0.0
2,19,1058725,8.0,7.0,14.0
3,20,529349,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,31.0,1.0,0.0
1,15,6.0,23.0,0.0
2,19,8.0,7.0,14.0
3,20,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,31.0
1,15,Released,6.0
2,19,Released,8.0
3,20,Released,4.0
4,10,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20
Status,,,,
Completed,0.0,0.0,14.0,0.0
Executing,1.0,23.0,7.0,0.0
Released,31.0,6.0,8.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20
0,Completed,0.0,0.0,14.0,0.0
1,Executing,1.0,23.0,7.0,0.0
2,Released,31.0,6.0,8.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20
0,Completed,0.0,0.0,14.0,0.0
1,Executing,1.0,23.0,7.0,0.0
2,Released,31.0,6.0,8.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()